## Step 1: Load & profile the Q2 2025 LA Metro Bike Share data


In [7]:
import pandas as pd
import numpy as np

RAW_PATH = "/workspaces/la-metro-bikeshare-insights/data/raw/metro-trips-2025-q2.csv"  # relative path from notebooks/

In [8]:
df = pd.read_csv(RAW_PATH)
display(df.head(10))
print("Shape:", df.shape)

,trip_id,duration,start_time,end_time,start_station,start_lat,start_lon,end_station,end_lat,end_lon,bike_id,plan_duration,trip_route_category,passholder_type,bike_type
0,497794246,1,4/1/2025 0:03,4/1/2025 0:04,4512,34.043812,-118.264969,4512,34.043812,-118.264969,19854,30,Round Trip,Monthly Pass,electric
1,497794374,5,4/1/2025 0:06,4/1/2025 0:11,4512,34.043812,-118.264969,3005,34.048500,-118.258537,19854,30,One Way,Monthly Pass,electric
2,497794444,6,4/1/2025 0:09,4/1/2025 0:15,4538,34.065430,-118.308823,4524,34.061630,-118.301193,15902,30,One Way,Monthly Pass,standard
3,497794638,9,4/1/2025 0:20,4/1/2025 0:29,3005,34.048500,-118.258537,4315,34.054031,-118.270920,19854,30,One Way,Monthly Pass,electric
4,497794649,7,4/1/2025 0:22,4/1/2025 0:29,3005,34.048500,-118.258537,3074,34.044170,-118.261169,29600,30,One Way,Monthly Pass,electric
5,497796120,35,4/1/2025 0:30,4/1/2025 1:05,4315,34.054031,-118.270920,4538,34.065430,-118.308823,19854,30,One Way,Monthly Pass,electric
6,497796095,20,4/1/2025 0:42,4/1/2025 1:02,3054,34.039219,-118.236488,3082,34.046520,-118.237411,15575,1,One Way,Walk-up,standard
7,497796174,29,4/1/2025 0:44,4/1/2025 1:13,4534,34.101860,-118.328110,4534,34.101860,-118.328110,23041,30,Round Trip,Monthly Pass,electric
8,497796298,42,4/1/2025 0:45,4/1/2025 1:27,4536,33.989079,-118.462257,4536,33.989079,-118.462257,5784,30,Round Trip,Monthly Pass,standard
9,497796273,26,4/1/2025 0:59,4/1/2025 1:24,3075,34.042110,-118.256187,3019,34.038609,-118.260857,20113,1,One Way,One Day Pass,standard


Shape: (123109, 15)


In [9]:
print("Column dtypes:")
print(df.dtypes)

print("\nNull counts (top 20):")
print(df.isna().sum().sort_values(ascending=False).head(20))


Column dtypes:
trip_id                  int64
duration                 int64
start_time              object
end_time                object
start_station            int64
start_lat              float64
start_lon              float64
end_station              int64
end_lat                float64
end_lon                float64
bike_id                  int64
plan_duration            int64
trip_route_category     object
passholder_type         object
bike_type               object
dtype: object

Null counts (top 20):
end_lat                3142
end_lon                3142
start_lon                 2
start_lat                 2
duration                  0
start_time                0
trip_id                   0
end_time                  0
start_station             0
end_station               0
bike_id                   0
plan_duration             0
trip_route_category       0
passholder_type           0
bike_type                 0
dtype: int64


In [10]:
for col in ["trip_route_category", "passholder_type", "bike_type"]:
    if col in df.columns:
        print(f"\nTop values in {col}:")
        print(df[col].value_counts(dropna=False).head(10))



Top values in trip_route_category:
trip_route_category
One Way       99214
Round Trip    23895
Name: count, dtype: int64

Top values in passholder_type:
passholder_type
Monthly Pass    79901
Walk-up         22635
Annual Pass     11873
One Day Pass     8700
Name: count, dtype: int64

Top values in bike_type:
bike_type
standard    76226
electric    46883
Name: count, dtype: int64


## Step 2: Parse datetimes & add basic validation fields

In [11]:
import pandas as pd
import numpy as np
from math import radians, sin, cos, asin, sqrt

NUMERIC_COLS = ["duration", "start_lat", "start_lon", "end_lat", "end_lon", "plan_duration"]
TIME_COLS = ["start_time", "end_time"]

def parse_dt(series: pd.Series) -> pd.Series:
    # Handles "4/1/25 00:03" style; errors -> NaT
    return pd.to_datetime(series, errors="coerce", infer_datetime_format=True)

def haversine_km(lat1, lon1, lat2, lon2):
    # Returns NaN if any coord is missing
    if any(pd.isna(v) for v in (lat1, lon1, lat2, lon2)):
        return np.nan
    R = 6371.0
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    c = 2 * asin(np.sqrt(a))
    return R * c

In [12]:
df_numeric = df.copy()
for col in NUMERIC_COLS:
    if col in df_numeric.columns:
        df_numeric[col] = pd.to_numeric(df_numeric[col], errors="coerce")

df_numeric[NUMERIC_COLS].head()


,duration,start_lat,start_lon,end_lat,end_lon,plan_duration
0,1,34.043812,-118.264969,34.043812,-118.264969,30
1,5,34.043812,-118.264969,34.048500,-118.258537,30
2,6,34.065430,-118.308823,34.061630,-118.301193,30
3,9,34.048500,-118.258537,34.054031,-118.270920,30
4,7,34.048500,-118.258537,34.044170,-118.261169,30


In [13]:
df_numeric["start_dt"] = parse_dt(df_numeric["start_time"])
df_numeric["end_dt"]   = parse_dt(df_numeric["end_time"])

print(df_numeric[["start_time", "end_time", "start_dt", "end_dt"]].head(5))
print("Time parse errors:", int(df_numeric["start_dt"].isna().sum() + df_numeric["end_dt"].isna().sum()))

/tmp/ipykernel_4551/2629514913.py:10: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.to_datetime(series, errors="coerce", infer_datetime_format=True)


      start_time       end_time            start_dt              end_dt
0  4/1/2025 0:03  4/1/2025 0:04 2025-04-01 00:03:00 2025-04-01 00:04:00
1  4/1/2025 0:06  4/1/2025 0:11 2025-04-01 00:06:00 2025-04-01 00:11:00
2  4/1/2025 0:09  4/1/2025 0:15 2025-04-01 00:09:00 2025-04-01 00:15:00
3  4/1/2025 0:20  4/1/2025 0:29 2025-04-01 00:20:00 2025-04-01 00:29:00
4  4/1/2025 0:22  4/1/2025 0:29 2025-04-01 00:22:00 2025-04-01 00:29:00
Time parse errors: 0


/tmp/ipykernel_4551/2629514913.py:10: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.to_datetime(series, errors="coerce", infer_datetime_format=True)


In [14]:
# Reported duration is in minutes in this dataset
df_numeric["duration_min_reported"] = df_numeric["duration"]

# Actual duration from timestamps (minutes)
df_numeric["duration_min_actual"] = (df_numeric["end_dt"] - df_numeric["start_dt"]).dt.total_seconds() / 60

# Helpful comparison field
df_numeric["duration_abs_diff_min"] = (df_numeric["duration_min_actual"] - df_numeric["duration_min_reported"]).abs()

df_numeric[["duration_min_reported", "duration_min_actual", "duration_abs_diff_min"]].describe()

,duration_min_reported,duration_min_actual,duration_abs_diff_min
count,123109.000000,123109.000000,123109.000000
mean,33.020941,36.871618,3.866533
std,101.222077,201.182757,139.253436
min,1.000000,0.000000,0.000000
25%,7.000000,7.000000,0.000000
50%,15.000000,15.000000,0.000000
75%,27.000000,27.000000,0.000000
max,1440.000000,18450.000000,17010.000000


In [15]:
df_numeric["distance_km"] = np.vectorize(haversine_km)(
    df_numeric["start_lat"], df_numeric["start_lon"],
    df_numeric["end_lat"], df_numeric["end_lon"]
)

# Speed (km/h); avoid divide-by-zero
df_numeric["speed_kmh"] = df_numeric["distance_km"] / (df_numeric["duration_min_actual"] / 60.0)
df_numeric.loc[(df_numeric["duration_min_actual"] <= 0) | (df_numeric["duration_min_actual"].isna()), "speed_kmh"] = np.nan

df_numeric[["distance_km", "duration_min_actual", "speed_kmh"]].describe()

,distance_km,duration_min_actual,speed_kmh
count,119965.000000,123109.000000,119914.000000
mean,1.535377,36.871618,6.105054
std,1.832241,201.182757,4.731297
min,0.000000,0.000000,0.000000
25%,0.388012,7.000000,1.221128
50%,0.989120,15.000000,6.412831
75%,2.086655,27.000000,9.561396
max,24.786499,18450.000000,128.925094


In [16]:
df_numeric["flag_time_parse_error"]        = df_numeric["start_dt"].isna() | df_numeric["end_dt"].isna()
df_numeric["flag_negative_or_zero_duration"]= df_numeric["duration_min_actual"] <= 0
df_numeric["flag_duration_mismatch_gt2"]   = df_numeric["duration_abs_diff_min"] > 2   # > 2 minutes difference
df_numeric["flag_speed_unrealistic"]       = df_numeric["speed_kmh"] > 45              # generous e-bike upper bound
df_numeric["flag_same_station_low_dist"]   = (df_numeric["start_station"] == df_numeric["end_station"]) & (df_numeric["distance_km"].fillna(0) < 0.05)

flag_cols = [
    "flag_time_parse_error","flag_negative_or_zero_duration",
    "flag_duration_mismatch_gt2","flag_speed_unrealistic","flag_same_station_low_dist"
]
df_numeric[flag_cols].mean().sort_values(ascending=False).rename("share_of_rows")


flag_same_station_low_dist        0.194096
flag_duration_mismatch_gt2        0.002071
flag_negative_or_zero_duration    0.000414
flag_speed_unrealistic            0.000008
flag_time_parse_error             0.000000
Name: share_of_rows, dtype: float64

In [17]:
total = len(df_numeric)
summary_rows = []
for col in flag_cols:
    cnt = int(df_numeric[col].sum())
    summary_rows.append({
        "flag": col,
        "count": cnt,
        "percent": round(100 * cnt / total, 2)
    })
summary_df = pd.DataFrame(summary_rows).sort_values("percent", ascending=False)
summary_df


,flag,count,percent
4,flag_same_station_low_dist,23895,19.41
2,flag_duration_mismatch_gt2,255,0.21
1,flag_negative_or_zero_duration,51,0.04
0,flag_time_parse_error,0,0.00
3,flag_speed_unrealistic,1,0.00


In [19]:
processed_cols = [
    "trip_id","start_dt","end_dt","duration_min_reported","duration_min_actual",
    "start_station","end_station","start_lat","start_lon","end_lat","end_lon",
    "distance_km","speed_kmh","trip_route_category","passholder_type","bike_type",
    "plan_duration","duration_abs_diff_min"
] + flag_cols

sample_out_path = "../data/processed/metro-trips-2025-q2_processed_sample.csv"
df_numeric[processed_cols].head(10000).to_csv(sample_out_path, index=False)
sample_out_path


'../data/processed/metro-trips-2025-q2_processed_sample.csv'

## Step 3: Apply cleaning rules & save the cleaned dataset

In [20]:
# Start from df_numeric from Step 2
df_clean = df_numeric.copy()

# Define filters
mask_valid = (
    (~df_clean["flag_time_parse_error"]) &
    (~df_clean["flag_negative_or_zero_duration"]) &
    (~df_clean["flag_speed_unrealistic"])
)

# Optionally remove same-station < 50m trips
# REMOVE_SAME_STATION_SHORT = True
# if REMOVE_SAME_STATION_SHORT:
#     mask_valid &= ~df_clean["flag_same_station_low_dist"]

# Apply
df_clean = df_clean[mask_valid].reset_index(drop=True)
print(f"Rows before: {len(df_numeric):,} → after cleaning: {len(df_clean):,}")

Rows before: 123,109 → after cleaning: 123,057


In [21]:
print("Null counts after cleaning:")
print(df_clean.isna().sum().sort_values(ascending=False).head(15))

Null counts after cleaning:
speed_kmh              3144
distance_km            3144
end_lon                3142
end_lat                3142
start_lon                 2
start_lat                 2
start_time                0
end_time                  0
start_station             0
trip_id                   0
duration                  0
bike_id                   0
end_station               0
plan_duration             0
trip_route_category       0
dtype: int64


In [22]:
full_clean_path = "../data/processed/metro-trips-2025-q2_cleaned.csv"
df_clean.to_csv(full_clean_path, index=False)
full_clean_path

'../data/processed/metro-trips-2025-q2_cleaned.csv'

In [23]:
print("Date range:", df_clean["start_dt"].min(), "→", df_clean["start_dt"].max())
print("\nPassholder type counts:")
print(df_clean["passholder_type"].value_counts())

print("\nBike type counts:")
print(df_clean["bike_type"].value_counts())


Date range: 2025-04-01 00:03:00 → 2025-06-30 23:55:00

Passholder type counts:
passholder_type
Monthly Pass    79878
Walk-up         22627
Annual Pass     11857
One Day Pass     8695
Name: count, dtype: int64

Bike type counts:
bike_type
standard    76191
electric    46866
Name: count, dtype: int64
